# Apply_ProposalModel <a class="tocSkip">

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime as dt
from tqdm.notebook import tqdm as tqdm_nb

sys.path.append('../bin')
import myutil
import dataset
from dataset import Holomem, Hololive
import model
from model import Word2VecModel, ProposalModel, ProposalUser

prj_path = myutil.PROJECT_PATH
prj_path

/Users/nyu/gdrive/learn/hololi2vec/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


PosixPath('/Users/nyu/gdrive/learn/hololi2vec')

<h1>目次<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#データセット＆Word2Vecモデル読み込み" data-toc-modified-id="データセット＆Word2Vecモデル読み込み-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>データセット＆Word2Vecモデル読み込み</a></span></li><li><span><a href="#提案モデル学習" data-toc-modified-id="提案モデル学習-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>提案モデル学習</a></span><ul class="toc-item"><li><span><a href="#提案モデルのハイパーパラメータ" data-toc-modified-id="提案モデルのハイパーパラメータ-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>提案モデルのハイパーパラメータ</a></span></li><li><span><a href="#コサイン類似度モデル" data-toc-modified-id="コサイン類似度モデル-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>コサイン類似度モデル</a></span><ul class="toc-item"><li><span><a href="#インスタンス生成" data-toc-modified-id="インスタンス生成-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>インスタンス生成</a></span></li><li><span><a href="#訓練セットによるモデル学習" data-toc-modified-id="訓練セットによるモデル学習-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>訓練セットによるモデル学習</a></span></li><li><span><a href="#テストセットによるモデル学習" data-toc-modified-id="テストセットによるモデル学習-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>テストセットによるモデル学習</a></span></li></ul></li><li><span><a href="#順序差減衰モデル" data-toc-modified-id="順序差減衰モデル-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>順序差減衰モデル</a></span><ul class="toc-item"><li><span><a href="#インスタンス生成" data-toc-modified-id="インスタンス生成-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>インスタンス生成</a></span></li><li><span><a href="#訓練セットによるモデル学習" data-toc-modified-id="訓練セットによるモデル学習-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>訓練セットによるモデル学習</a></span></li><li><span><a href="#テストセットによるモデル学習" data-toc-modified-id="テストセットによるモデル学習-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>テストセットによるモデル学習</a></span></li></ul></li><li><span><a href="#表現ヒストリーの構造" data-toc-modified-id="表現ヒストリーの構造-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>表現ヒストリーの構造</a></span></li></ul></li></ul></div>

※ツイートテキストを含むような出力をするセルはGitHubプッシュ時にリセットして消してある

# データセット＆Word2Vecモデル読み込み
Hololiveクラスを使い，データセットを準備し，`05_Apply_Word2Vec`で訓練したモデルファイルを読み込む，

In [3]:
hololive_members = pd.read_csv(Path(prj_path, 'data/hololive_members.csv'))
hololive_members.head()

,twitter,name,generation
0,tokino_sora,ときのそら,0
1,robocosan,ロボ子さん,0
2,sakuramiko35,さくらみこ,0
3,suisei_hosimati,星街すいせい,0
4,yozoramel,夜空メル,1


In [4]:
hololive_userIds = hololive_members['twitter'].values
hololive_userIds

array(['tokino_sora', 'robocosan', 'sakuramiko35', 'suisei_hosimati',
       'yozoramel', 'shirakamifubuki', 'natsuiromatsuri', 'akaihaato',
       'akirosenthal', 'minatoaqua', 'nakiriayame', 'yuzukichococh',
       'murasakishionch', 'oozorasubaru', 'ookamimio', 'nekomataokayu',
       'inugamikorone', 'shiranuiflare', 'shiroganenoel', 'houshoumarine',
       'usadapekora', 'uruharushia', 'amanekanatach', 'kiryucoco',
       'tsunomakiwatame', 'tokoyamitowa', 'himemoriluna', 'yukihanalamy',
       'momosuzunene', 'shishirobotan', 'omarupolka'], dtype=object)

In [5]:
%%time
hololive = Hololive(userIds=hololive_userIds, n_tweet=3200, fname='hololive')

CPU times: user 11.9 s, sys: 356 ms, total: 12.3 s
Wall time: 12.4 s


In [6]:
train_df, test_df = hololive.split_data(date=dt(2021, 1, 1, 0, 0, 0), save=True, verbose=True)

[dataset] 以下パスに分かち書き結果とツイートデータを保存しました
[dataset] 分かち書き結果（訓練セット）: /Users/nyu/gdrive/learn/hololi2vec/data/wakatigaki/hololive_3200_20210101.txt
[dataset] 訓練セット: /Users/nyu/gdrive/learn/hololi2vec/data/dataset/hololive_3200_20210101_train.tsv
[dataset] テストセット: /Users/nyu/gdrive/learn/hololi2vec/data/dataset/hololive_3200_20210101_test.tsv


In [ ]:
hololive.train_df.head()

In [8]:
hololive.train_df.shape

(41764, 3)

In [ ]:
hololive.test_df.head()

In [10]:
hololive.test_df.shape

(15836, 3)

In [11]:
train_session_df, test_session_df = hololive.create_session_df(train_test=[True, True], save=True, verbose=True)

100%|██████████| 15836/15836 [00:01<00:00, 15491.71it/s]


[dataset] 以下パスに訓練セットのセッションデータを保存しました
[dataset] /Users/nyu/gdrive/learn/hololi2vec/data/session/hololive_3200_20210101_train_session.csv
[dataset] 以下パスにテストセットのセッションデータを保存しました
[dataset] /Users/nyu/gdrive/learn/hololi2vec/data/session/hololive_3200_20210101_test_session.csv


In [ ]:
hololive.train_session_df.head()

In [13]:
hololive.train_session_df.shape

(661189, 4)

In [ ]:
hololive.test_session_df.head()

In [15]:
hololive.test_session_df.shape

(225919, 4)

ここまでの処理により，ツイートデータを2021年1月1日を境界として訓練・テストセットに分割し，分かち書き結果txtファイルと単語単位のセッションデータ形式csvファイルの作成までできた．

次に，このデータセット（分かち書き結果txtファイル）を使用して訓練したWord2Vecモデルを読み込む．

In [16]:
w2v_model_v1 = Word2VecModel(hololive=hololive, model_name='v1', verbose=True)

[model] 以下パスからWord2Vecモデルを読み込みました
[model] /Users/nyu/gdrive/learn/hololi2vec/model/word2vec/hololive_v1.model


In [17]:
for k, v in vars(w2v_model_v1).items():
    print(k, '\t', v)

model_path 	 /Users/nyu/gdrive/learn/hololi2vec/model/word2vec/hololive_v1.model
loaded_model 	 True
sentences_path 	 /Users/nyu/gdrive/learn/hololi2vec/data/wakatigaki/hololive_3200_20210101.txt
verbose 	 True
model 	 Word2Vec(vocab=7825, size=50, alpha=0.025)
params 	 {'window': 17, 'size': 50, 'sample': 0.01, 'negative': 50, 'min_count': 5, 'epochs': 30}


- モデルは以下のハイパーパラメータ設定で学習されている（他のハイパーパラメータも固定値でいくつか弄っている）．
- 各ハイパーパラメータの詳細は`05_Apply_Word2Vec`の`2. Word2Vecモデル訓練`を参照されたい．

In [18]:
w2v_model_v1.show_params()

[model] Word2Vec ハイパーパラメータ
[model] window 17
[model] size 50
[model] sample 0.01
[model] negative 50
[model] min_count 5
[model] epochs 30


訓練セットとモデルがそれぞれもつ単語数は以下の通り

In [19]:
print('Train', len(set(hololive.test_session_df['word'])))
print('Word2Vec', len(w2v_model_v1.words))

Train 17992
Word2Vec 7825


このWord2Vecモデルの詳しい分析結果は`05_Apply_Word2Vec`の`3. モデル分析`を参照されたい

# 提案モデル学習
- 提案モデルそのもののクラスである`model.ProposalModel`と，提案モデルにおけるユーザークラスである`model.ProposalUser`を使い，提案モデルの学習を行う．
- Word2Vecモデルについては「訓練」と呼び，提案モデルについては「学習」と区別して呼ぶことについては，提案モデルが訓練データセットを使って処理することが「学習」の方がニュアンスが似てると考えたからである．
    - 訓練と学習は同一のものとして扱われることが多いが，ここでは分けることにしている．
    - 提案モデルは，訓練データセットを使って，各ユーザのツイート（セッション）表現とユーザ表現を構築・更新するのみで，パラメータを調整して汎化性能を上げることをしているわけではないため，訓練とは言い難い．
- 提案モデルの詳細は **https://www.slideshare.net/okamoto-laboratory/ss-239612151** を参照されたい（やや古い発表でデータセットの整形方法修正やモデルコードのバグ修正をこれ以後実施しており，最新の結果（今後公開される卒論）とは実験結果や最適ハイパーパラメータ設定が一部異なる）．
    - 上は提案モデルを使った推薦システムとしての発表だが，今回のプロジェクトは「セッションデータと似たような構造をしているツイートデータに対する提案モデル（アイテム分散表現の階層化・集約演算）の適用」をテーマとしており，提案モデルの別方向の応用案といえる．
    - セッションデータの詳細は`03_Prepare_Dataset`の`4. セッションデータ作成`を参照されたい．

In [20]:
type(hololive), type(w2v_model_v1)

(dataset.Hololive, model.Word2VecModel)

In [21]:
for k, v in vars(w2v_model_v1).items():
    print(k, '\t', v)

model_path 	 /Users/nyu/gdrive/learn/hololi2vec/model/word2vec/hololive_v1.model
loaded_model 	 True
sentences_path 	 /Users/nyu/gdrive/learn/hololi2vec/data/wakatigaki/hololive_3200_20210101.txt
verbose 	 True
model 	 Word2Vec(vocab=7825, size=50, alpha=0.025)
params 	 {'window': 17, 'size': 50, 'sample': 0.01, 'negative': 50, 'min_count': 5, 'epochs': 30}


- `Hololive`クラスのインスタンス（訓練・テストセット分割，分かち書きファイル作成済み）をデータセットとして与え，`ProposalModel`のインスタンスを生成する．
- `Word2VecModel`クラスのインスタンス（訓練済み）も与える．提案システムはWord2Vec（Item2Vec）をベースとしており，Word2Vecモデルが学習する分散表現の質に依存する．
- インスタンス生成時には提案モデルのハイパーパラメータ設定も辞書で渡す（詳細は次節で述べる）．

## 提案モデルのハイパーパラメータ
- 提案モデルは多くのハイパーパラメータを持つが，卒論に向けた実験により明らかに強い傾向が見られた設定は固定してしまう．
- しかし，次の3つについてはデータセットによって異なる傾向になる可能性が高く，`ProposalModel`に与えることで値を変更できるようにしてある
    - ツイート（セッション）表現構築タイプ: コサイン類似度と順序差減衰の2つの方法を提案している．
    - 崩壊定数: 順序差減衰によるツイート表現構築時に，重み決定のために使用される指数減衰関数の崩壊定数である．ツイート表現構築タイプがコサイン類似度の場合は与える必要がない．
    - コンテキストセッション数: ユーザ表現構築時に考慮する過去ツイート（セッション）数である．
- 改めて，提案モデルの詳細は **https://www.slideshare.net/okamoto-laboratory/ss-239612151** を参照されたい．

In [22]:
prop_cos_params = dict(tweet_construct_type='cos', user_context=5)
prop_odd_params = dict(tweet_construct_type='odd', tweet_lambda=0.7, user_context=5)

**[各ハイパーパラメータ設定値について]**

- ツイート（セッション）表現構築タイプ: どちらが良いかはまだ明らかになってないので両方試す．
- 崩壊定数: セッションデータにおいては0.7が良い傾向にあったため，とりあえず0.7にしておく．
- コンテキストセッション数: 全く予想がつかないため，とりあえず決め打ちで5にする（セッションデータでは3と5では5の方がやや良い傾向にあった）．

## コサイン類似度モデル
まず，ツイート表現構築タイプを「コサイン類似度」としたモデル（コサイン類似度モデル）を試す．

### インスタンス生成

In [23]:
%%time
prop_cos = ProposalModel(hololive=hololive, w2v_model=w2v_model_v1, params=prop_cos_params)

CPU times: user 14.5 s, sys: 725 ms, total: 15.2 s
Wall time: 15.4 s


インスタンス生成時点（モデル初期化時点）でのインスタンス変数は次の通り

**[ProposalModel]**
- `w2v_model`: `Word2VecModel`クラスのインスタンス（訓練セットによる訓練済み）
- `train`: 訓練セット（セッションデータ形式）（`Hololive.train_session_df`）
- `test`: テストセット（セッションデータ形式）（`Hololive.test_session_df`）
- `params`: ハイパーパラメータ設定
- `userIds`: ユーザIDリスト（`Hololive.userIds`）
- `users`: 提案モデルユーザリスト（`userIds`の各IDをキーとし，値は対応する`ProposalUser`クラスのインスタンス）

In [24]:
for k, v in vars(prop_cos).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'user_reps'}:
        print(len(v))
    elif k in {'train', 'test'}:
        print(v.shape)
    else:
        print(v)

[w2v_model] (<class 'model.Word2VecModel'>)
[train] (<class 'pandas.core.frame.DataFrame'>)
(661189, 4)
[test] (<class 'pandas.core.frame.DataFrame'>)
(225919, 4)
[params] (<class 'dict'>)
{'tweet_construct_type': 'cos', 'user_context': 5}
[userIds] (<class 'numpy.ndarray'>)
['tokino_sora' 'robocosan' 'sakuramiko35' 'suisei_hosimati' 'yozoramel'
 'shirakamifubuki' 'natsuiromatsuri' 'akaihaato' 'akirosenthal'
 'minatoaqua' 'nakiriayame' 'yuzukichococh' 'murasakishionch'
 'oozorasubaru' 'ookamimio' 'nekomataokayu' 'inugamikorone'
 'shiranuiflare' 'shiroganenoel' 'houshoumarine' 'usadapekora'
 'uruharushia' 'amanekanatach' 'kiryucoco' 'tsunomakiwatame'
 'tokoyamitowa' 'himemoriluna' 'yukihanalamy' 'momosuzunene'
 'shishirobotan' 'omarupolka']
[users] (<class 'dict'>)
{'tokino_sora': <model.ProposalUser object at 0x12f314460>, 'robocosan': <model.ProposalUser object at 0x12f3147f0>, 'sakuramiko35': <model.ProposalUser object at 0x132904c70>, 'suisei_hosimati': <model.ProposalUser object at

**[ProposalUser]**
このクラスは`dataset.Holomem`を継承している
- `userId`, `name`, `generation`, `n_tweet`, `load`, `verbose`, `preview`, `tweets`, `df`: Holomemクラスのインスタンス変数
- `tweetIds`: ツイート（セッション）表現が構築されたツイートIDのリスト
- `tweet_reps`: `tweetIds`の各IDに対応するツイート表現
- `user_rep`: ユーザ表現．未構築状態であるため`None`になっている．
- `proposal_model`: `ProposalUser`インスタンスとリンクしている`ProposalModel`インスタンス
- `w2v_model`: `Word2VecModel`クラスのインスタンス（`proposal_model.w2v_model`）
- `params`: ハイパーパラメータ設定（`proposal_model.params`）

以下では，例としてシオンちゃんの`ProposalUser`インスタンスを示す．

In [25]:
for k, v in vars(prop_cos.users['murasakishionch']).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'tweets', 'tweetIds'}:
        print(len(v))
    elif k in {'df'}:
        print(len(v.shape))
    elif k in {'tweet_reps'}:
        print(len(v), v)
    else:
        print(v)

[userId] (<class 'str'>)
murasakishionch
[name] (<class 'str'>)
紫咲シオン
[generation] (<class 'set'>)
{'2'}
[n_tweet] (<class 'int'>)
3200
[load] (<class 'bool'>)
True
[verbose] (<class 'bool'>)
False
[preview] (<class 'bool'>)
False
[tweets_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.pkl
[tweets_df_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.tsv
[tweets] (<class 'list'>)
3200
[df] (<class 'pandas.core.frame.DataFrame'>)
2
[tweetIds] (<class 'list'>)
1
[tweet_reps] (<class 'dict'>)
0 {}
[user_rep] (<class 'NoneType'>)
None
[proposal_model] (<class 'model.ProposalModel'>)
[w2v_model] (<class 'model.Word2VecModel'>)
[params] (<class 'dict'>)
{'tweet_construct_type': 'cos', 'user_context': 5}


### 訓練セットによるモデル学習
- `ProposalModel.learn()`は`isTrain`という引数を唯一持っている．
- 訓練セットによる学習時は`isTrain=True`とする．
- テストセットによる学習時は，`isTrain=False`にすることで全ユーザの各表現の記録`reps_history`をつける（分析に使用）．

In [26]:
%%time
prop_cos.learn(isTrain=True)

  0%|          | 0/661189 [00:00<?, ?it/s]

CPU times: user 20.1 s, sys: 3.01 s, total: 23.1 s
Wall time: 20.8 s


訓練セットによるモデル学習後のインスタンス変数は次の通り

**[ProposalModel]**（更新されたor新たに追加されたインスタンス変数のみ）
- `users`: 値の`ProposalUser`が更新されている（後述）
- `user_reps`: `userIds`の各IDに対応するユーザ表現

In [27]:
for k, v in vars(prop_cos).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'user_reps'}:
        print(len(v), v['murasakishionch'])
    elif k in {'train', 'test'}:
        print(v.shape)
    else:
        print(v)

[w2v_model] (<class 'model.Word2VecModel'>)
[train] (<class 'pandas.core.frame.DataFrame'>)
(661189, 4)
[test] (<class 'pandas.core.frame.DataFrame'>)
(225919, 4)
[params] (<class 'dict'>)
{'tweet_construct_type': 'cos', 'user_context': 5}
[userIds] (<class 'numpy.ndarray'>)
['tokino_sora' 'robocosan' 'sakuramiko35' 'suisei_hosimati' 'yozoramel'
 'shirakamifubuki' 'natsuiromatsuri' 'akaihaato' 'akirosenthal'
 'minatoaqua' 'nakiriayame' 'yuzukichococh' 'murasakishionch'
 'oozorasubaru' 'ookamimio' 'nekomataokayu' 'inugamikorone'
 'shiranuiflare' 'shiroganenoel' 'houshoumarine' 'usadapekora'
 'uruharushia' 'amanekanatach' 'kiryucoco' 'tsunomakiwatame'
 'tokoyamitowa' 'himemoriluna' 'yukihanalamy' 'momosuzunene'
 'shishirobotan' 'omarupolka']
[users] (<class 'dict'>)
{'tokino_sora': <model.ProposalUser object at 0x12f314460>, 'robocosan': <model.ProposalUser object at 0x12f3147f0>, 'sakuramiko35': <model.ProposalUser object at 0x132904c70>, 'suisei_hosimati': <model.ProposalUser object at

**[ProposalUser]**（更新されたor新たに追加されたインスタンス変数のみ）
- `tweetIds`: ツイート（セッション）表現が構築されたツイートIDのリスト．後述するが，すべてのツイートのツイート表現が構築されるとは限らない．
- `tweet_reps`: `tweetIds`の各IDに対応するツイート表現．`tweetIds`が初期値として`[-1]`を持ち，このリストにツイートIDを追加していくため，`tweet_reps`のサイズは`tweetIds`より1小さくなる．
- `user_rep`: ユーザ表現．`None`ではなくなっている．
- `proposal_model`: `ProposalUser`インスタンスとリンクしている`ProposalModel`インスタンス．更新された`ProposalModel`のインスタンス変数は先に述べた通り．

In [28]:
for k, v in vars(prop_cos.users['murasakishionch']).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'tweets', 'tweetIds'}:
        print(len(v))
    elif k in {'df'}:
        print(v.shape)
    elif k in {'tweet_reps'}:
        print(len(v), v[prop_cos.users['murasakishionch'].tweetIds[1]].shape)
    else:
        print(v)

[userId] (<class 'str'>)
murasakishionch
[name] (<class 'str'>)
紫咲シオン
[generation] (<class 'set'>)
{'2'}
[n_tweet] (<class 'int'>)
3200
[load] (<class 'bool'>)
True
[verbose] (<class 'bool'>)
False
[preview] (<class 'bool'>)
False
[tweets_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.pkl
[tweets_df_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.tsv
[tweets] (<class 'list'>)
3200
[df] (<class 'pandas.core.frame.DataFrame'>)
(2573, 3)
[tweetIds] (<class 'list'>)
2196
[tweet_reps] (<class 'dict'>)
2195 (50,)
[user_rep] (<class 'numpy.ndarray'>)
[-1.64357107e-01 -2.78379509e-01  2.54413876e-02  1.47594815e-01
  5.50623744e-02  1.75366636e-01 -2.09519979e-01  2.52754207e-02
 -6.17412964e-01 -4.25447198e-01  2.02606629e-01  9.90378275e-02
 -1.63933176e-01 -3.22292460e-02 -1.23567048e-01 -3.18737937e-02
  1.43136963e-01  1.41755026e-01  2.44117208e-01  2.05201308e-01
  1.73548285

`tweetIds`（先頭の`-1`除く），と`tweet_reps.keys()`は必ず一致する

In [29]:
assert prop_cos.users['murasakishionch'].tweetIds[1:] == list(prop_cos.users['murasakishionch'].tweet_reps.keys())

- 与えたデータセットに含まれるツイート数と，実際に構築されたツイート表現数は必ずしも一致するとは限らない．
- あるツイートに含まれる単語のいずれにおいても，訓練セットに含まれていないがために単語表現が取得できなかった場合，そのツイートのツイート表現が構築できない．
- そのようなケースは非常に限られていることが予想されるが，起こる可能性はゼロではないため，このような僅かな差が生じている．

In [30]:
print('n_tweet\t\t', len(hololive.train_df[hololive.train_df['userId'] == 'murasakishionch']))
print('n_tweet_rep\t', len(prop_cos.users['murasakishionch'].tweetIds))

n_tweet		 2200
n_tweet_rep	 2196


ツイート・ユーザ表現の次元数はいずれも`w2v_model.params['size']`と一致する．

In [31]:
assert prop_cos.w2v_model.params['size'] == prop_cos.users['murasakishionch'].tweet_reps[prop_cos.users['murasakishionch'].tweetIds[-1]].shape[0]
assert prop_cos.w2v_model.params['size'] == prop_cos.users['murasakishionch'].user_rep.shape[0]

### テストセットによるモデル学習
`isTrain=False`にすることで分析のための全ユーザの各表現の記録`reps_history`をつける．

処理するログ数は訓練セットの1/3程度しかないが，記録をつける処理が追加される影響で実行時間は約2.5倍になっている．

In [32]:
%%time
prop_cos.learn(isTrain=False)

  0%|          | 0/225919 [00:00<?, ?it/s]

CPU times: user 51.6 s, sys: 2.82 s, total: 54.4 s
Wall time: 54.5 s


テストセットによるモデル学習後のインスタンス変数は次の通り

**[ProposalModel]**（更新されたor新たに追加されたインスタンス変数のみ）
- `users`: 値の`ProposalUser`が更新されている
- `user_reps`: `userIds`の各IDに対応するユーザ表現
- `reps_history`: 表現ヒストリー．詳細は2.4で述べる．

In [33]:
for k, v in vars(prop_cos).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'user_reps'}:
        print('(len, element): {}'.format((len(v), v['murasakishionch'])))
    elif k in {'train', 'test'}:
        print('shape:', v.shape)
    elif k in {'reps_history'}:
        print('sample_user:' ,'murasakishionch', v[-1]['murasakishionch'])
    else:
        print(v)

[w2v_model] (<class 'model.Word2VecModel'>)
[train] (<class 'pandas.core.frame.DataFrame'>)
shape: (661189, 4)
[test] (<class 'pandas.core.frame.DataFrame'>)
shape: (225919, 4)
[params] (<class 'dict'>)
{'tweet_construct_type': 'cos', 'user_context': 5}
[userIds] (<class 'numpy.ndarray'>)
['tokino_sora' 'robocosan' 'sakuramiko35' 'suisei_hosimati' 'yozoramel'
 'shirakamifubuki' 'natsuiromatsuri' 'akaihaato' 'akirosenthal'
 'minatoaqua' 'nakiriayame' 'yuzukichococh' 'murasakishionch'
 'oozorasubaru' 'ookamimio' 'nekomataokayu' 'inugamikorone'
 'shiranuiflare' 'shiroganenoel' 'houshoumarine' 'usadapekora'
 'uruharushia' 'amanekanatach' 'kiryucoco' 'tsunomakiwatame'
 'tokoyamitowa' 'himemoriluna' 'yukihanalamy' 'momosuzunene'
 'shishirobotan' 'omarupolka']
[users] (<class 'dict'>)
{'tokino_sora': <model.ProposalUser object at 0x12f314460>, 'robocosan': <model.ProposalUser object at 0x12f3147f0>, 'sakuramiko35': <model.ProposalUser object at 0x132904c70>, 'suisei_hosimati': <model.Proposal

**[ProposalUser]**（更新されたor新たに追加されたインスタンス変数のみ）
- `tweetIds`: ツイート（セッション）表現が構築されたツイートIDのリスト．訓練セットによるモデル学習後の`tweetIds`に追加される形になっている．
- `tweet_reps`: `tweetIds`の各IDに対応するツイート表現．
- `user_rep`: ユーザ表現．
- `proposal_model`: `ProposalUser`インスタンスとリンクしている`ProposalModel`インスタンス．更新・追加された`ProposalModel`のインスタンス変数は先に述べた通り．

In [34]:
for k, v in vars(prop_cos.users['murasakishionch']).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'tweets', 'tweetIds'}:
        print('len:', len(v))
    elif k in {'df'}:
        print('shape:', v.shape)
    elif k in {'tweet_reps'}:
        print('(len, element): {}'.format((len(v), v[prop_cos.users['murasakishionch'].tweetIds[1]])))
    else:
        print(v)

[userId] (<class 'str'>)
murasakishionch
[name] (<class 'str'>)
紫咲シオン
[generation] (<class 'set'>)
{'2'}
[n_tweet] (<class 'int'>)
3200
[load] (<class 'bool'>)
True
[verbose] (<class 'bool'>)
False
[preview] (<class 'bool'>)
False
[tweets_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.pkl
[tweets_df_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.tsv
[tweets] (<class 'list'>)
len: 3200
[df] (<class 'pandas.core.frame.DataFrame'>)
shape: (2573, 3)
[tweetIds] (<class 'list'>)
len: 2394
[tweet_reps] (<class 'dict'>)
(len, element): (2393, array([-0.41033912, -0.31465462,  0.00629656, -0.1206122 ,  0.32318568,
        0.25138915, -0.1192783 ,  0.0105895 , -0.3910103 , -0.38833833,
        0.2626191 , -0.04111688, -0.16912755,  0.41238344, -0.08083791,
       -0.05192978, -0.06790527, -0.01574949,  0.22362402,  0.24202012,
       -0.18749912,  0.53588337, -0.07133744, -0.23397805

訓練セットによる学習後と同様に，`tweetIds`（先頭の`-1`除く）と`tweet_reps.keys()`は必ず一致する．

In [35]:
assert prop_cos.users['murasakishionch'].tweetIds[1:] == list(prop_cos.users['murasakishionch'].tweet_reps.keys())

与えたデータセットに含まれるツイート数と，実際に構築されたツイート表現数の差は以下の通り．

In [36]:
n_tweet = len(hololive.train_df[hololive.train_df['userId'] == 'murasakishionch']
             ) + len(hololive.test_df[hololive.test_df['userId'] == 'murasakishionch'])

print('n_tweet (train+test)\t', n_tweet)
print('n_tweet_rep (train+test)', len(prop_cos.users['murasakishionch'].tweetIds))
print('n_tweet - n_tweet_rep\t', n_tweet - len(prop_cos.users['murasakishionch'].tweetIds))

n_tweet (train+test)	 2398
n_tweet_rep (train+test) 2394
n_tweet - n_tweet_rep	 4


差が訓練セットによる学習後から変化していないことから，テストセットに含まれるシオンちゃんのツイートに対してはすべてのツイート表現が構築できたといえる．

## 順序差減衰モデル
- ツイート表現構築タイプを「順序差減衰」としたモデル（順序差減衰モデル）も同様にインスタンス生成とモデル学習を行う．
- 各セルに対する説明はコサイン類似度モデルの場合と同様であるため基本省略する．

### インスタンス生成

In [37]:
prop_odd_params

{'tweet_construct_type': 'odd', 'tweet_lambda': 0.7, 'user_context': 5}

In [38]:
%%time
prop_odd = ProposalModel(hololive=hololive, w2v_model=w2v_model_v1, params=prop_odd_params)

CPU times: user 15.6 s, sys: 2.31 s, total: 17.9 s
Wall time: 18.3 s


In [39]:
for k, v in vars(prop_odd).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'user_reps'}:
        print(len(v), v['murasakishionch'])
    elif k in {'train', 'test'}:
        print(v.shape)
    else:
        print(v)

[w2v_model] (<class 'model.Word2VecModel'>)
[train] (<class 'pandas.core.frame.DataFrame'>)
(661189, 4)
[test] (<class 'pandas.core.frame.DataFrame'>)
(225919, 4)
[params] (<class 'dict'>)
{'tweet_construct_type': 'odd', 'tweet_lambda': 0.7, 'user_context': 5}
[userIds] (<class 'numpy.ndarray'>)
['tokino_sora' 'robocosan' 'sakuramiko35' 'suisei_hosimati' 'yozoramel'
 'shirakamifubuki' 'natsuiromatsuri' 'akaihaato' 'akirosenthal'
 'minatoaqua' 'nakiriayame' 'yuzukichococh' 'murasakishionch'
 'oozorasubaru' 'ookamimio' 'nekomataokayu' 'inugamikorone'
 'shiranuiflare' 'shiroganenoel' 'houshoumarine' 'usadapekora'
 'uruharushia' 'amanekanatach' 'kiryucoco' 'tsunomakiwatame'
 'tokoyamitowa' 'himemoriluna' 'yukihanalamy' 'momosuzunene'
 'shishirobotan' 'omarupolka']
[users] (<class 'dict'>)
{'tokino_sora': <model.ProposalUser object at 0x11c75a8e0>, 'robocosan': <model.ProposalUser object at 0x108d785b0>, 'sakuramiko35': <model.ProposalUser object at 0x1460b10d0>, 'suisei_hosimati': <model.P

In [40]:
for k, v in vars(prop_odd.users['murasakishionch']).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'tweets', 'tweetIds'}:
        print(len(v))
    elif k in {'df'}:
        print(len(v.shape))
    elif k in {'tweet_reps'}:
        print(len(v), v)
    else:
        print(v)

[userId] (<class 'str'>)
murasakishionch
[name] (<class 'str'>)
紫咲シオン
[generation] (<class 'set'>)
{'2'}
[n_tweet] (<class 'int'>)
3200
[load] (<class 'bool'>)
True
[verbose] (<class 'bool'>)
False
[preview] (<class 'bool'>)
False
[tweets_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.pkl
[tweets_df_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.tsv
[tweets] (<class 'list'>)
3200
[df] (<class 'pandas.core.frame.DataFrame'>)
2
[tweetIds] (<class 'list'>)
1
[tweet_reps] (<class 'dict'>)
0 {}
[user_rep] (<class 'NoneType'>)
None
[proposal_model] (<class 'model.ProposalModel'>)
[w2v_model] (<class 'model.Word2VecModel'>)
[params] (<class 'dict'>)
{'tweet_construct_type': 'odd', 'tweet_lambda': 0.7, 'user_context': 5}


### 訓練セットによるモデル学習

順序差減衰モデルはコサイン類似度モデルと異なり，順序差ごとの重みが固定されているので計算コストが小さい（計算時間は7秒ほど短い）．

In [41]:
%%time
prop_odd.learn(isTrain=True)

  0%|          | 0/661189 [00:00<?, ?it/s]

CPU times: user 13.1 s, sys: 2.87 s, total: 16 s
Wall time: 13.8 s


In [42]:
for k, v in vars(prop_odd).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'user_reps'}:
        print(len(v), v['murasakishionch'])
    elif k in {'train', 'test'}:
        print(v.shape)
    else:
        print(v)

[w2v_model] (<class 'model.Word2VecModel'>)
[train] (<class 'pandas.core.frame.DataFrame'>)
(661189, 4)
[test] (<class 'pandas.core.frame.DataFrame'>)
(225919, 4)
[params] (<class 'dict'>)
{'tweet_construct_type': 'odd', 'tweet_lambda': 0.7, 'user_context': 5}
[userIds] (<class 'numpy.ndarray'>)
['tokino_sora' 'robocosan' 'sakuramiko35' 'suisei_hosimati' 'yozoramel'
 'shirakamifubuki' 'natsuiromatsuri' 'akaihaato' 'akirosenthal'
 'minatoaqua' 'nakiriayame' 'yuzukichococh' 'murasakishionch'
 'oozorasubaru' 'ookamimio' 'nekomataokayu' 'inugamikorone'
 'shiranuiflare' 'shiroganenoel' 'houshoumarine' 'usadapekora'
 'uruharushia' 'amanekanatach' 'kiryucoco' 'tsunomakiwatame'
 'tokoyamitowa' 'himemoriluna' 'yukihanalamy' 'momosuzunene'
 'shishirobotan' 'omarupolka']
[users] (<class 'dict'>)
{'tokino_sora': <model.ProposalUser object at 0x11c75a8e0>, 'robocosan': <model.ProposalUser object at 0x108d785b0>, 'sakuramiko35': <model.ProposalUser object at 0x1460b10d0>, 'suisei_hosimati': <model.P

In [43]:
for k, v in vars(prop_odd.users['murasakishionch']).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'tweets', 'tweetIds'}:
        print(len(v))
    elif k in {'df'}:
        print(v.shape)
    elif k in {'tweet_reps'}:
        print(len(v), v[prop_odd.users['murasakishionch'].tweetIds[1]].shape)
    else:
        print(v)

[userId] (<class 'str'>)
murasakishionch
[name] (<class 'str'>)
紫咲シオン
[generation] (<class 'set'>)
{'2'}
[n_tweet] (<class 'int'>)
3200
[load] (<class 'bool'>)
True
[verbose] (<class 'bool'>)
False
[preview] (<class 'bool'>)
False
[tweets_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.pkl
[tweets_df_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.tsv
[tweets] (<class 'list'>)
3200
[df] (<class 'pandas.core.frame.DataFrame'>)
(2573, 3)
[tweetIds] (<class 'list'>)
2196
[tweet_reps] (<class 'dict'>)
2195 (50,)
[user_rep] (<class 'numpy.ndarray'>)
[-0.29450607 -0.50609656 -0.06723322  0.2738281   0.10910782  0.3159867
 -0.29856065  0.02055269 -1.01290672 -0.70859104  0.40440639  0.13980059
 -0.28362237  0.03058577 -0.21755623 -0.05631731  0.14647349  0.23919974
  0.42528588  0.47115206 -0.00925899  0.81448274  0.07532489 -0.47347966
 -0.17326204  0.1725173   0.00175947  0.964610

In [44]:
assert prop_odd.users['murasakishionch'].tweetIds[1:] == list(prop_odd.users['murasakishionch'].tweet_reps.keys())

In [45]:
print('n_tweet\t\t', len(hololive.train_df[hololive.train_df['userId'] == 'murasakishionch']))
print('n_tweet_rep\t', len(prop_odd.users['murasakishionch'].tweetIds))

n_tweet		 2200
n_tweet_rep	 2196


In [46]:
assert prop_odd.w2v_model.params['size'] == prop_odd.users['murasakishionch'].tweet_reps[prop_odd.users['murasakishionch'].tweetIds[-1]].shape[0]
assert prop_odd.w2v_model.params['size'] == prop_odd.users['murasakishionch'].user_rep.shape[0]

### テストセットによるモデル学習

In [47]:
%%time
prop_odd.learn(isTrain=False)

  0%|          | 0/225919 [00:00<?, ?it/s]

CPU times: user 53.4 s, sys: 9.13 s, total: 1min 2s
Wall time: 1min 4s


In [48]:
for k, v in vars(prop_odd).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'user_reps'}:
        print('(len, element): {}'.format((len(v), v['murasakishionch'])))
    elif k in {'train', 'test'}:
        print('shape:', v.shape)
    elif k in {'reps_history'}:
        print('sample_user:' ,'murasakishionch', v[-1]['murasakishionch'])
    else:
        print(v)

[w2v_model] (<class 'model.Word2VecModel'>)
[train] (<class 'pandas.core.frame.DataFrame'>)
shape: (661189, 4)
[test] (<class 'pandas.core.frame.DataFrame'>)
shape: (225919, 4)
[params] (<class 'dict'>)
{'tweet_construct_type': 'odd', 'tweet_lambda': 0.7, 'user_context': 5}
[userIds] (<class 'numpy.ndarray'>)
['tokino_sora' 'robocosan' 'sakuramiko35' 'suisei_hosimati' 'yozoramel'
 'shirakamifubuki' 'natsuiromatsuri' 'akaihaato' 'akirosenthal'
 'minatoaqua' 'nakiriayame' 'yuzukichococh' 'murasakishionch'
 'oozorasubaru' 'ookamimio' 'nekomataokayu' 'inugamikorone'
 'shiranuiflare' 'shiroganenoel' 'houshoumarine' 'usadapekora'
 'uruharushia' 'amanekanatach' 'kiryucoco' 'tsunomakiwatame'
 'tokoyamitowa' 'himemoriluna' 'yukihanalamy' 'momosuzunene'
 'shishirobotan' 'omarupolka']
[users] (<class 'dict'>)
{'tokino_sora': <model.ProposalUser object at 0x11c75a8e0>, 'robocosan': <model.ProposalUser object at 0x108d785b0>, 'sakuramiko35': <model.ProposalUser object at 0x1460b10d0>, 'suisei_hosim

In [49]:
for k, v in vars(prop_odd.users['murasakishionch']).items():
    print('[{}] ({})'.format(k, type(v)))
    if k in {'tweets', 'tweetIds'}:
        print('len:', len(v))
    elif k in {'df'}:
        print('shape:', v.shape)
    elif k in {'tweet_reps'}:
        print('(len, element): {}'.format((len(v), v[prop_odd.users['murasakishionch'].tweetIds[1]])))
    else:
        print(v)

[userId] (<class 'str'>)
murasakishionch
[name] (<class 'str'>)
紫咲シオン
[generation] (<class 'set'>)
{'2'}
[n_tweet] (<class 'int'>)
3200
[load] (<class 'bool'>)
True
[verbose] (<class 'bool'>)
False
[preview] (<class 'bool'>)
False
[tweets_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.pkl
[tweets_df_path] (<class 'pathlib.PosixPath'>)
/Users/nyu/gdrive/learn/hololi2vec/data/tweets_murasakishionch_3200.tsv
[tweets] (<class 'list'>)
len: 3200
[df] (<class 'pandas.core.frame.DataFrame'>)
shape: (2573, 3)
[tweetIds] (<class 'list'>)
len: 2394
[tweet_reps] (<class 'dict'>)
(len, element): (2393, array([-0.8311162 , -0.62652993,  0.04734085, -0.2653988 ,  0.65774375,
        0.49875754, -0.22026092, -0.00372992, -0.7687181 , -0.79203373,
        0.5371815 , -0.10637884, -0.32669318,  0.8833764 , -0.14401518,
       -0.0953752 , -0.13733052, -0.04542314,  0.44243574,  0.5145321 ,
       -0.38134876,  1.0976605 , -0.11751367, -0.42890152

In [50]:
assert prop_odd.users['murasakishionch'].tweetIds[1:] == list(prop_odd.users['murasakishionch'].tweet_reps.keys())

In [51]:
n_tweet = len(hololive.train_df[hololive.train_df['userId'] == 'murasakishionch']
             ) + len(hololive.test_df[hololive.test_df['userId'] == 'murasakishionch'])

print('n_tweet (train+test)\t', n_tweet)
print('n_tweet_rep (train+test)', len(prop_odd.users['murasakishionch'].tweetIds))
print('n_tweet - n_tweet_rep\t', n_tweet - len(prop_odd.users['murasakishionch'].tweetIds))

n_tweet (train+test)	 2398
n_tweet_rep (train+test) 2394
n_tweet - n_tweet_rep	 4


## 表現ヒストリーの構造
- 表現ヒストリーとは，`ProposalModel.learn(isTrain=False)`としたとき，各時点での各ユーザの各表現を記録したものである．
- そのままではベクトル次元数が大きく把握が難しいため，PCA・RP・t-SNEで2or3次元に落として可視化するなどしてから分析に使用する．
- `ProposalModel.learn(isTrain=False)`の実行後，インスタンス変数`reps_history`が新たに追加されており，これが表現ヒストリーである

表現ヒストリーは3層構造の辞書になっている
- 第1層: セッションデータログのインデックス: セッションデータの各ログのインデックスを処理した時点での各ユーザの各表現を格納．初期化時点では`-1`をキーとして，訓練セットによる学習終了時点の各ユーザの各表現が格納されている．
- 第2層: ユーザID（Twitter ID）: インデックス時点での各表現が格納されている．
- 第3層: ツイート表現`tweet_rep`とユーザ表現`user_rep`: 先述した通り，初期化時点で，訓練セットによる訓練セットによる学習終了時点の各ユーザの各表現が格納されているため，最初から常にツイート表現とユーザ表現のどちらも持っている．

コサイン類似度モデル

In [52]:
print('idx:', len(prop_cos.reps_history.keys()), list(prop_cos.reps_history.keys())[:10])
print('userId:', len(prop_cos.reps_history[-1].keys()), list(prop_cos.reps_history[-1].keys())[:10])
print('rep:', prop_cos.reps_history[-1]['murasakishionch'])

idx: 225920 [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
userId: 31 ['tokino_sora', 'robocosan', 'sakuramiko35', 'suisei_hosimati', 'yozoramel', 'shirakamifubuki', 'natsuiromatsuri', 'akaihaato', 'akirosenthal', 'minatoaqua']
rep: {'tweet_rep': array([-0.38087392, -0.21508718, -0.09984877, -0.03285884,  0.2614746 ,
        0.22743541, -0.17077759,  0.01259382, -0.5729439 , -0.40227866,
        0.3897619 ,  0.03146386, -0.1634658 ,  0.26495594, -0.09742281,
        0.00969832, -0.05913102, -0.03112137,  0.10232247,  0.3502683 ,
       -0.08506773,  0.7119093 , -0.03284251, -0.01126166, -0.09972657,
        0.09520157, -0.02933645,  0.52338314, -0.20653194, -0.35440797,
        0.0184336 ,  0.31089997,  0.29125273,  0.3388365 ,  0.12781462,
       -0.05506627,  0.52174246,  0.08055086, -0.4262746 ,  0.08692914,
       -0.12909333, -0.05127875, -0.23139285, -0.09177579, -0.12955011,
       -0.16408654,  0.20534672,  0.13826327, -0.3504176 , -0.349572  ],
      dtype=float32), 'user_rep': array([-1.643

順序差減衰モデル

In [53]:
print('idx:', len(prop_odd.reps_history.keys()), list(prop_odd.reps_history.keys())[:10])
print('userId:', len(prop_odd.reps_history[-1].keys()), list(prop_odd.reps_history[-1].keys())[:10])
print('rep:', prop_odd.reps_history[-1]['murasakishionch'])

idx: 225920 [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
userId: 31 ['tokino_sora', 'robocosan', 'sakuramiko35', 'suisei_hosimati', 'yozoramel', 'shirakamifubuki', 'natsuiromatsuri', 'akaihaato', 'akirosenthal', 'minatoaqua']
rep: {'tweet_rep': array([-0.74262273, -0.42835703, -0.21733063, -0.00269198,  0.5155693 ,
        0.49934924, -0.28069597,  0.03369913, -1.077214  , -0.78469646,
        0.77886355,  0.06708199, -0.24234988,  0.5885814 , -0.17740619,
        0.05847525, -0.1372669 , -0.075897  ,  0.20354709,  0.68731403,
       -0.16311191,  1.4291284 , -0.01480927, -0.09825921, -0.23265176,
        0.16595866, -0.09837691,  0.97361994, -0.46388927, -0.8097959 ,
        0.08928843,  0.6052445 ,  0.6457179 ,  0.6555813 ,  0.21372291,
       -0.02363081,  1.0803633 ,  0.13568413, -0.7789119 ,  0.16541545,
       -0.34550405, -0.1624274 , -0.46683592, -0.09292251, -0.21462744,
       -0.3413351 ,  0.38975057,  0.2028848 , -0.7406682 , -0.7036699 ],
      dtype=float32), 'user_rep': array([-0.294

コサイン類似度モデルと順序差減衰モデルでは同じ時点での同じユーザでも各表現が異なっており，正常にパラメータ設定を区別して各表現が構築されている．